In [ ]:
!pip install langgraph langchain_core langchain_groq langchain_huggingface langchain_chroma

In [24]:
from typing import TypedDict,List,Dict,Any
from langgraph.graph import StateGraph,START,END
from langchain_groq import ChatGroq
from google.colab import userdata
import os
os.environ['GROQ_API_KEY']=userdata.get('GROQ_API_KEY')
from pydantic import BaseModel,Field,dataclasses


In [41]:
from typing import TypedDict,List,Dict,Any
from langgraph.graph import StateGraph,START,END
from langchain_groq import ChatGroq
from google.colab import userdata
import os
os.environ['GROQ_API_KEY']=userdata.get('GROQ_API_KEY')
from pydantic import BaseModel,Field,dataclasses

class GraphState(BaseModel):
  query:str
  plan:List[Dict[str,str]] = Field(default_factory=list)
  next_index:int = 0
  results:dict[str,Any] = Field(default_factory=dict)
  final_answer:str = ''
  next: str = '' # Add 'next' field to GraphState

class Step(BaseModel):
  action:str=Field(description='Action needs to be performed')
  query:str=Field(description='Query to use for action')

class Plan(BaseModel):
  steps:List[Step]=Field(description='A list of steps to complete the user task')

def planner(graph:GraphState):

  user_query=graph.query

  llm=ChatGroq(model='llama-3.1-8b-instant').with_structured_output(Plan)

  prompt=f"""You are a excellent planner who will be planning sequentially the user's query to accomplish it. Each plan
             Each plan must have action and query for it. For complex query you may use multiple plans to accomplish it
             User query : {user_query}"""

  response=llm.invoke(prompt)

  # Convert list of Step objects to list of dictionaries
  plan_list_of_dicts = [step.model_dump() for step in response.steps]

  return {'plan':plan_list_of_dicts,
          'next_index':0,
          'results':{},
          'query':user_query,
          'final_answer':'',
          'next':''} # Initialize 'next' in planner

def exectutor(graph:GraphState):

  current_plan=graph.plan
  current_index=graph.next_index

  if current_index>= len(current_plan):
      print('All tasks completed')
      return {}

  mock_responses = {
        "capital of canada": "The capital of Canada is Ottawa.",
        "population of ottawa": "The population of Ottawa is approximately 1.07 million.",
        "gdp of france": "The GDP of France is approximately 3.1 trillion USD.",
        "currency of france": "The currency of France is the Euro (€)."
    }

  current_step=current_plan[current_index]

  action=current_step['action']
  query=current_step['query']


  print(f"--- Executor is running step {current_index+1}/{len(current_plan)}: {action} with query '{query}' ---")

  action_output=mock_responses.get(query.lower(),'No result')

  graph.results[query]=action_output

  return {'results': graph.results,
          'next_index':current_index+1}

def manager(graph:GraphState):

  if graph.next_index<len(graph.plan):
    return {"next": "continue_plan"} # Return dictionary for conditional edge

  else:
    return {'next': 'synthesize'} # Return dictionary for conditional edge


def synthesise(graph:GraphState):

  print('Total no of performaed actions', len(graph.results) )

  llm=ChatGroq(model='llama-3.1-8b-instant')

  results_str= '\n\n'.join(f'Step for "{q}" resulted in "{r}" ' for q,r in graph.results.items())
  prompt=f"""You are an answer synthsize. For user query, it is divided into subtasks and necessary actions and its output are performed already .
            Input_query:{graph.query}
            Subtasks resuls :{results_str}
            Provide on output in concise.
          """

  final_answer = llm.invoke(prompt)

  return {'final_answer':final_answer.content}

agent=StateGraph(GraphState)

agent.add_node('planner',planner)
agent.add_node('executor',exectutor)
agent.add_node('manager',manager)
agent.add_node('synthesize',synthesise)

agent.set_entry_point('planner')
agent.add_edge('planner','executor')
agent.add_edge('executor','manager')
agent.add_conditional_edges('manager',
                                  lambda x: x.next, # Use dot notation to access 'next'
                                  {'continue_plan':'executor',
                                   'synthesize':'synthesize'})
agent.add_edge('synthesize',END)

app=agent.compile()

In [42]:
app.invoke({'query':
            'What is currency of france and capital of Canada and GDP of france'})

--- Executor is running step 1/3: search with query 'currency of France' ---
--- Executor is running step 2/3: search with query 'capital of Canada' ---
--- Executor is running step 3/3: search with query 'GDP of France' ---
Total no of performaed actions 3


{'query': 'What is currency of france and capital of Canada and GDP of france',
 'plan': [{'action': 'search', 'query': 'currency of France'},
  {'action': 'search', 'query': 'capital of Canada'},
  {'action': 'search', 'query': 'GDP of France'}],
 'next_index': 3,
 'results': {'currency of France': 'The currency of France is the Euro (€).',
  'capital of Canada': 'The capital of Canada is Ottawa.',
  'GDP of France': 'The GDP of France is approximately 3.1 trillion USD.'},
 'final_answer': 'The currency of France is the Euro (€), the capital of Canada is Ottawa, and the GDP of France is approximately 3.1 trillion USD.',
 'next': 'synthesize'}